### Probabilistic forecasting using [GluonTS](https://ts.gluon.ai/index.html): Ethereum example
[Probabilistic forecasting](https://en.wikipedia.org/wiki/Probabilistic_forecasting), rather than providing a single point prediction, provides a probability distribution as the outcome.
To do this we shall be using the [GluonTS - Probabilistic Time Series Modeling](https://ts.gluon.ai/index.html) package. This notebook is heavily based on the [Quick Start Tutorial](https://ts.gluon.ai/tutorials/forecasting/quick_start_tutorial.html) and the [Extended Forecasting Tutorial](https://ts.gluon.ai/tutorials/forecasting/extended_tutorial.html) that are both provided with the package.

# 1- Import Libs

In [ ]:
!pip install gluonts

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 12})
#plt.rcParams["figure.figsize"] = (12, 4)
from datetime import date
from gluonts.dataset.common import ListDataset

import math
from math import ceil
# Error libs
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import max_error
from sklearn.metrics import median_absolute_error
#from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance
from sklearn.metrics import mean_tweedie_deviance
from math import sqrt

# 2- Import Dataset

In [ ]:
train = pd.read_csv("../input/g-research-crypto-forecasting/train.csv")
# extract the data corresponding to ethereum (Asset_ID = 6)
ethereum = train.query("Asset_ID == 6").reset_index(drop = True)
ethereum['timestamp'] = pd.to_datetime(ethereum['timestamp'], unit='s')
ethereum = ethereum.set_index('timestamp')
# extract the "High" value at 9:00 a.m. daily
indexer_9am = ethereum.index.indexer_at_time('9:00:00')
values_at_9am = ethereum.iloc[indexer_9am]
high_values_at_9am = values_at_9am[["High"]]
# take a look
high_values_at_9am

# 3- Create Model

We shall forecast three months (30 days) worth of data, based on the 180 days prior to the start of the forecasting period

In [ ]:
freq = "1D"             # the frequency of our data, here daily
context_length    = 180 # train on this number of days
prediction_length =  30 # predict these many days, these are removed from the end of the training data

Convert our dataframe into a GluonTS dataset

In [ ]:
data_list = [{"start": "2018-01-01 09:00:00", "target": high_values_at_9am[c].values} for c in high_values_at_9am.columns]
train_ds  = ListDataset(data_iter=data_list,freq="1D")

### Training
Here we use the GulonTS’s pre-built [feedforward neural network estimator](https://ts.gluon.ai/api/gluonts/gluonts.model.simple_feedforward.html) `SimpleFeedForwardEstimator` in conjunction with a [trainer](https://ts.gluon.ai/api/gluonts/gluonts.mx.trainer.html). This feedforward network can be [substituted for a recurrent neural network (RNN)](https://ts.gluon.ai/tutorials/forecasting/extended_tutorial.html#From-feedforward-to-RNN), such as the  [DeepAREstimator](https://ts.gluon.ai/api/gluonts/gluonts.model.deepar.html).

In [ ]:
from gluonts.model.simple_feedforward  import SimpleFeedForwardEstimator
# to use the DeepAREstimator 
# from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.distribution.student_t import StudentTOutput
from gluonts.mx import Trainer

estimator = SimpleFeedForwardEstimator(num_hidden_dimensions=[60],
                                       freq=freq,
                                       context_length=context_length,
                                       prediction_length=prediction_length,
                                       distr_output=StudentTOutput(),
                                       trainer=Trainer(epochs=50,
                                                       learning_rate=1e-3,
                                                       num_batches_per_epoch=100,
                                                       patience=10))

predictor = estimator.train(train_ds)
print("Done")

### Prediction
We shall now make (here 2000) predictions using our model

In [ ]:
from gluonts.evaluation import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_ds,  # dataset
    predictor=predictor,  # predictor
    num_samples=2000,  # number of sample paths we want for evaluation
)

forecasts = list(forecast_it)
tss = list(ts_it)

### Visualization
And finally we shall now plot our probabilistic forecast (here with the 50% and 90% prediction intervals shown) along with the ground truth values

In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = context_length + prediction_length
    prediction_intervals = (50.0, 95.0)
    legend = ["ground truth", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(18, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='cadetblue')
    plt.axvline(forecast_entry.start_date, color='g', lw=1) # end of train dataset
    plt.grid(which="major")
    plt.legend(legend, loc="upper left")
    plt.show();
    
plot_prob_forecasts(tss[0], forecasts[0])

We can extract the mean and median values of our forecast as follows:

In [ ]:
# first entry of the forecast list
forecast_entry = forecasts[0]
print(f"Mean of the future window:\n {forecast_entry.mean}")
print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.5)}")

In [ ]:
ControlTrain=high_values_at_9am.iloc[-30:,:]
ControlTrain["Prediction"]=forecast_entry.mean
ControlTrain

### Evaluate the results

In [ ]:
class errors:
      
    def mean_absolute_percentage(y, y_hat): 
        
        #y_true=y_true.replace(0,0.001)
        #y_pred=y_pred.replace(0,0.001)
        y, y_hat = np.array(y), np.array(y_hat)
        return np.mean(np.abs((y - y_hat) / y)) * 100    
    
    def mean_absolute_percentage_notabs(y, y_hat): 
        
        #y_true=y_true.replace(0,0.001)
        #y_pred=y_pred.replace(0,0.001)
        #y, y_hat = np.array(y), np.array(y_hat)
        return np.mean(((y - y_hat) / y)) * 100    
    
    def bias(y,yhat):    
        y, y_hat = np.array(y), np.array(y_hat)
        return np.sum(y - y_hat)
        
    def smape(y_true, y_pred):
        denominator = (np.abs(y_true) + np.abs(y_pred))
        diff = np.abs(y_true - y_pred) / denominator
        diff[denominator == 0] = 0.0
        return 200 * np.mean(diff)
    def performance_metrics (y,yhat):
        #print("y shape : ",y.shape , " , y_hat shape : ", yhat.shape)
        try:
            '''
            R² is the ratio between how good our model is vs how good is the naive mean model.
            '''
            R_2 = round(r2_score(y, yhat),3)                
        except:
            R_2 = "EXC" 
            
        '''
        Adjusted R2 required Regressor Number
        '''

        try:
            '''
            Explained variance regression score function
            '''
            EVS = round(explained_variance_score(y, yhat),3)
        except:
            EVS = "EXC"  
            
        try:
            '''
            Mean absolute error regression loss
            it’s not that sensitive to outliers as mean square error. 
            MAE is more robust (less sensitive to outliers) than MSE
            The measure gives an idea of the magnitude of the error, but no idea of the direction (e.g. over or under predicting).
            '''
            MAE = int(mean_absolute_error(y, yhat))                   
        except:
            MAE = "EXC" 
            
        try:
            '''
            Mean squared error regression loss
            MSE basically measures average squared error of our predictions. For each point, it calculates square difference between the predictions and the target and then average those values.
            The higher this value, the worse the model is. It is never negative, since we’re squaring the individual prediction-wise errors before summing them, but would be zero for a perfect model .
            Advantage: Useful if we have unexpected values that we should care about. Vey high or low value that we should pay attention.
            Disadvantage: If we make a single very bad prediction, the squaring will make the error even worse and it may skew the metric towards overestimating the model’s badness. That is a particularly problematic behaviour if we have noisy data (that is, data that
            for whatever reason is not entirely reliable) — even a “perfect” model may have a high MSE in that situation, so it becomes hard to judge how well the model is performing. On the other hand, if all the errors are small, or rather, smaller than 1, than the
            opposite effect is felt: we may underestimate the model’s badness.
            Note that if we want to have a constant prediction the best one will be the mean value of the target values. It can be found by setting the derivative of our total error with respect to that constant to zero, and find it from this equation.
            '''
            MSE = int(mean_squared_error(y, yhat))                   
        except:
            MSE = "EXC" 
            
        try:    
            '''
            RMSE
            '''
            RMSE = round(mean_squared_error(y, yhat, squared=False),2)
        except: 
            RMSE = "EXC" 
            
        try:
            '''
            The bias is defined as the average error.
            '''
            BIAS = int (bias(y,yhat))
        except:
            BIAS = "EXC" 
            
        try:
            
            MAPE= round(errors.mean_absolute_percentage(y, yhat),2)         
        except:
            MAPE= "EXC" 
            
        try:
            '''  	
            max_error metric calculates the maximum residual error.
            '''            
            MAXE = round (max_error(y, yhat),2)
        except:            
            MAXE ="EXC"
            
        try:            
            '''
            Mean squared logarithmic error regression loss
            '''
            MSLE = round(mean_squared_log_error(y, yhat),2)
        except:            
            MSLE ="EXC"
        try:            
            '''
            Median absolute error regression loss
            '''
            MDAE = round(median_absolute_error(y, yhat),2)
        except:            
            MDAE ="EXC"
        try:            
            '''
            Mean Poisson Deviance
            '''
            MPD = 0 # round(mean_poisson_deviance(y, yhat),2)
        except:            
            MPD ="EXC"
            
        try:            
            '''
            Mean Gamma Deviance
            '''
            MGD = round(mean_gamma_deviance(y, yhat),2)
        except:            
            MGD ="EXC"
            
        try:            
            '''
            Mean Tweedie Deviance
            '''
            MTD = round(mean_tweedie_deviance(y, yhat),2)
        except:            
            MTD ="EXC"
        try:            
            '''
            SMAPE
            '''
            SMAPE = round(smape(y, yhat),2)
        except:            
            SMAPE ="EXC"     
        
        return R_2, EVS, MAE, MSE, RMSE, BIAS, MAPE, MTD, MGD, MPD, MSLE, MDAE, MAXE, SMAPE
    
    
    
    def results_dict (y,yhat):
        
        R_2, EVS, MAE, MSE, RMSE, BIAS, MAPE, MTD, MGD, MPD, MSLE, MDAE, MAXE,SMAPE= errors.performance_metrics (y,yhat)
        
        return {'r2' : R_2 , 'evs' : EVS , 'mae' : MAE , 'mse' : MSE , 'mape' : MAPE, 'rmse' : RMSE, 'bias' : BIAS, 'mtd' : MTD, 'mgd' : MGD, 'mpd' : MPD, 'msle' : MSLE, 'mdae' : MDAE, 'maxe' : MAXE, "smape": SMAPE}
    
    
    
    def print_results(y,y_hat):
        
        R_2, EVS, MAE, MSE, RMSE, BIAS, MAPE, MTD, MGD, MPD, MSLE, MDAE, MAXE,SMAPE = errors.performance_metrics (y,y_hat)
        print ('----------------------------------------------------------------')
        print ('                        MODEL ERROR METRICS                     ')
        print ('----------------------------------------------------------------')
        print ("- R2 Score                       (R^2) = " + str (R_2) )
        print ("- Explained Variance Score       (EVS) = " + str (EVS) )
        print ("- Mean Absolute Error            (MAE) = " + str (MAE) )
        print ("- Mean Squared Error             (MSE) = " + str (MSE) )
        print ("- Mean Absolute Percentage Error (MAPE)= %" + str (MAPE))
        print ("- Root Mean Squared Error        (RMSE)= " + str (RMSE))
        print ("- BIAS...SUM(y-yhat)             (BIAS)= " + str (BIAS))
        print ("- Mean Tweedie Deviance          (MTD) = " + str (MTD) )
        print ("- Mean Gamma Deviance            (MGD) = " + str (MGD) )
        print ("- Mean Poisson Deviance          (MPD) = " + str (MPD) )
        print ("- Mean squared logarithmic Error (MSLE)= " + str (MSLE))     
        print ("- Median absolute Error          (MDAE)= " + str (MDAE))        
        print ("- Maximum Residual Error         (MAXE)= " + str (MAXE))
        print ("-  S Mean Absolute Percentage Error (SMAPE)= %" + str (SMAPE))
        
        print ('----------------------------------------------------------------')
        print ('\n')

In [ ]:
errors.print_results(ControlTrain.High,ControlTrain.Prediction)